In [1]:
import time
import json
import numpy as np
import pandas as pd
import covasim as cv
from pyDOE import lhs
from src.data import get_regional_data
from src.interventions import get_sampling_interventions

Covasim 2.0.3 (2021-03-11) — © 2021 by IDM


# Data Generation

## Initial Configuration

Once the dataset is calibrated, we would like to use it to predict possible future evolutions of the pandemic.

Given that *Covasim* needs a warm-up period to reach a stabler pandemic condition, we will proceed by generating the dataset in this way:
* we run a simulation for a total of *zones_starting_day $+$ num_zones $\cdot$ time_interval days*
* we discard the first *zones_starting_day* days (251, in our case), meaning that we will collect data from 01/11/2020 on, which is when the evolution of the pandemic started having a behaviour more similar to the one we have right now (i.e., after the summer period)
* we aggregate the data on a time window of size *2 $\cdot$ time_interval* (6 weeks, in our case), in which we consider the first half of the data as "inputs", and the last half as "outputs"
  * right in the middle of these windows there is a zone change
  * it will be used as input of the surrogate model together with the other data

In this way, we aim at maximizing the number of samples that we can get from each run, while trying at the same time to cover the data space in a (hopefully) almost uniform way.

In [2]:
num_zones = 8
time_interval = 21
zones_starting_day = 251

We can now retrieve the parameters' configuration that we obtained from the calibration step and set the length of the simulation from the previously set values.

In [3]:
with open('../res/parameters.json', 'r') as json_file:
    j = json.load(json_file)

intervention_params = j['intervention_params']
initial_params = j['initial_params']
initial_params['n_days'] = zones_starting_day + num_zones * time_interval
df = get_regional_data(4.46e6 / initial_params['pop_size'])

initial_params

{'pop_type': 'hybrid',
 'location': 'italy',
 'start_day': '2020-02-24',
 'pop_size': 400000,
 'pop_infected': 720,
 'n_beds_hosp': 1481.6,
 'n_beds_icu': 57.84,
 'quar_period': 14,
 'verbose': 0,
 'n_days': 419}

## Latin-Hypercube Sampling

We have *4* possible categories for the zones, and each run of the simulator will consider a sequence of *8* zone changes. Thus, the total number of possible sequences is $4^8 = 65536$, which, if we consider an average of 2 minutes for each run, will end the generation phase after 90 days.

This is clearly not desirable, thus we rely on *Latin-Hypercube Sampling* to generate a subset of *300* sequences which, however, is supposed to cover the sequences space almost uniformly. Again, if we consider an average of 2 minutes for each run, will end the generation phase after 10 hours.

In [4]:
np.random.seed(42)
samples = lhs(n=num_zones, samples=200)
samples = pd.DataFrame(samples)
for i in range(num_zones):
    samples[i] = samples[i].map(lambda v: int(4 * v)).map({0: 'W', 1: 'Y', 2: 'O', 3: 'R'})
samples

,0,1,2,3,4,5,6,7
0,Y,R,W,Y,R,Y,Y,Y
1,Y,O,O,Y,R,Y,R,Y
2,Y,O,W,O,R,O,W,O
3,O,W,W,W,Y,R,R,Y
4,R,Y,O,O,R,O,R,W
...,...,...,...,...,...,...,...,...
195,O,Y,O,R,Y,O,O,W
196,Y,R,Y,R,W,R,O,Y
197,O,O,O,R,O,W,W,W
198,Y,Y,O,O,O,W,W,R


We can start the generation script, from which we will collect the data about *hospitalized* individuals and cumulative *diagnosed* cases and *deaths*.

In [5]:
data = []
for idx, zones in samples.iterrows():
    print(f'Generating samples for simulation {idx + 1:0{len(str(samples.shape[0]))}}/{samples.shape[0]}', end='')
    start_time = time.time()
    intervs = get_sampling_interventions(zones, intervention_params, time_interval)
    sim = cv.Sim(pars={**initial_params, 'rand_seed': idx}, interventions=intervs, datafile=df)
    sim.run()
    # concatenate data in an array of shape (num_days, 4)
    result = np.concatenate((
        sim.results['n_severe'].values + sim.results['n_critical'].values,
        sim.results['cum_diagnoses'].values,
        sim.results['cum_deaths'].values
    )).reshape(3, -1).transpose()
    # retrieve data about last num_zones * time_interval and flatten it to get num_zones rows
    result = result[-num_zones * time_interval:].reshape(num_zones, -1)
    # get data about two subsequent zones with respective zone colors
    data += [np.concatenate((inp, out, zones[z:z + 2])) for z, (inp, out), in enumerate(zip(result[:-1], result[1:]))]
    print(f' -- elapsed time: {time.time() - start_time:.4}s')

Generating samples for simulation 001/200 -- elapsed time: 258.3s
Generating samples for simulation 002/200 -- elapsed time: 213.1s
Generating samples for simulation 003/200 -- elapsed time: 243.3s
Generating samples for simulation 004/200 -- elapsed time: 231.8s
Generating samples for simulation 005/200 -- elapsed time: 227.7s
Generating samples for simulation 006/200 -- elapsed time: 240.5s
Generating samples for simulation 007/200 -- elapsed time: 242.9s
Generating samples for simulation 008/200 -- elapsed time: 261.4s
Generating samples for simulation 009/200 -- elapsed time: 245.4s
Generating samples for simulation 010/200 -- elapsed time: 214.8s
Generating samples for simulation 011/200 -- elapsed time: 252.1s
Generating samples for simulation 012/200 -- elapsed time: 231.8s
Generating samples for simulation 013/200 -- elapsed time: 228.8s
Generating samples for simulation 014/200 -- elapsed time: 235.9s
Generating samples for simulation 015/200 -- elapsed time: 248.8s
Generating

## Saving the Dataset

We can now collect the data into a pandas dataframe and store it in a *csv* file to be used to train the surrogate model.

In [6]:
pd.options.display.max_columns = 2 * time_interval + 2

columns = [f'{c}_{d}' for d in range(0, 2 * time_interval) for c in ['hosp', 'diag', 'dead']]
data = pd.DataFrame(data, columns=columns + ['init_zone', 'actuated_zone'])
data.to_csv('../res/dataset.csv', index=False)
data

,hosp_0,diag_0,dead_0,hosp_1,diag_1,dead_1,hosp_2,diag_2,dead_2,hosp_3,diag_3,dead_3,hosp_4,diag_4,dead_4,hosp_5,diag_5,dead_5,hosp_6,diag_6,dead_6,hosp_7,...,diag_35,dead_35,hosp_36,diag_36,dead_36,hosp_37,diag_37,dead_37,hosp_38,diag_38,dead_38,hosp_39,diag_39,dead_39,hosp_40,diag_40,dead_40,hosp_41,diag_41,dead_41,init_zone,actuated_zone
0,98.0,7137.0,420.0,106.0,7255.0,421.0,107.0,7344.0,424.0,116.0,7446.0,424.0,125.0,7560.0,424.0,134.0,7649.0,426.0,137.0,7725.0,428.0,151.0,...,10570.0,535.0,180.0,10613.0,538.0,173.0,10632.0,543.0,167.0,10679.0,545.0,151.0,10723.0,550.0,143.0,10759.0,552.0,135.0,10778.0,553.0,Y,R
1,224.0,9389.0,463.0,219.0,9524.0,470.0,223.0,9652.0,474.0,227.0,9763.0,477.0,229.0,9885.0,482.0,222.0,9998.0,488.0,224.0,10066.0,494.0,230.0,...,11150.0,572.0,61.0,11235.0,573.0,68.0,11294.0,573.0,69.0,11357.0,576.0,76.0,11416.0,576.0,78.0,11444.0,577.0,83.0,11525.0,578.0,R,W
2,119.0,10800.0,556.0,109.0,10817.0,558.0,100.0,10840.0,559.0,91.0,10866.0,560.0,85.0,10893.0,560.0,75.0,10919.0,565.0,68.0,10929.0,567.0,69.0,...,12958.0,601.0,192.0,13126.0,606.0,186.0,13297.0,611.0,194.0,13435.0,616.0,201.0,13564.0,619.0,199.0,13710.0,621.0,196.0,13822.0,627.0,W,Y
3,94.0,11575.0,578.0,96.0,11674.0,580.0,106.0,11774.0,580.0,111.0,11847.0,582.0,117.0,11952.0,583.0,124.0,12037.0,585.0,132.0,12117.0,588.0,146.0,...,15130.0,681.0,203.0,15192.0,683.0,188.0,15228.0,688.0,157.0,15261.0,697.0,152.0,15288.0,700.0,144.0,15309.0,701.0,138.0,15324.0,706.0,Y,R
4,193.0,13887.0,633.0,198.0,14057.0,635.0,199.0,14202.0,640.0,206.0,14371.0,643.0,211.0,14484.0,648.0,211.0,14593.0,649.0,209.0,14662.0,655.0,208.0,...,15616.0,730.0,31.0,15664.0,730.0,31.0,15705.0,731.0,34.0,15743.0,731.0,34.0,15779.0,732.0,35.0,15807.0,734.0,36.0,15840.0,735.0,R,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,128.0,10591.0,541.0,121.0,10614.0,543.0,104.0,10628.0,547.0,89.0,10634.0,551.0,75.0,10654.0,556.0,76.0,10666.0,558.0,69.0,10686.0,559.0,64.0,...,10809.0,566.0,5.0,10821.0,566.0,5.0,10831.0,566.0,8.0,10840.0,566.0,9.0,10847.0,566.0,9.0,10854.0,566.0,10.0,10868.0,566.0,R,O
1396,15.0,10763.0,565.0,14.0,10764.0,565.0,13.0,10765.0,565.0,11.0,10765.0,565.0,10.0,10768.0,565.0,9.0,10770.0,565.0,8.0,10771.0,565.0,7.0,...,10996.0,570.0,19.0,11006.0,571.0,20.0,11010.0,571.0,20.0,11013.0,571.0,20.0,11016.0,571.0,18.0,11017.0,571.0,13.0,11017.0,572.0,O,R
1397,10.0,10875.0,566.0,13.0,10891.0,566.0,16.0,10903.0,566.0,16.0,10916.0,566.0,17.0,10928.0,566.0,18.0,10936.0,566.0,23.0,10943.0,566.0,23.0,...,11139.0,574.0,10.0,11158.0,575.0,12.0,11174.0,575.0,13.0,11203.0,575.0,14.0,11220.0,575.0,15.0,11231.0,575.0,14.0,11244.0,576.0,R,W
1398,11.0,11018.0,573.0,6.0,11020.0,574.0,5.0,11022.0,574.0,4.0,11024.0,574.0,4.0,11024.0,574.0,4.0,11029.0,574.0,4.0,11033.0,574.0,4.0,...,11639.0,584.0,41.0,11668.0,584.0,46.0,11689.0,585.0,49.0,11715.0,585.0,47.0,11729.0,586.0,46.0,11750.0,588.0,36.0,11759.0,593.0,W,O
